In [40]:
!ls


2019-07-08.csv                  connect.ssh
BigQuery_LondonCrimes           create long term file....ipynb
Cat feeder.ipynb                fastai_gcp.rtf
Long Term Analysis .ipynb       improved.csv
Update the dataset script.ipynb local plotly test.ipynb
Weight.csv                      mictosoftcompetition notes.rtf
Weight2019_07_12.csv            pi-iot-project.json
Weight2019_07_18.csv            temp-plot.html
Weight2019_07_24.csv            training-data-analyst


In [41]:
import os
import pandas as pd
from pandas.io import gbq
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "pi-iot-project.json"

# Instantiates a client
bigquery_client = bigquery.Client()

In [42]:
improved_old=pd.read_csv('improved.csv')
improved_old.tail()

,time,val,Day,Hour,delta,eaten,given,eaten_cum_Day,given_cum_Day
8696,2019-07-23 23:34:06,16.0,2019-07-23,23:34:06,0.0,0.0,0.0,75.0,51.0
8697,2019-07-23 23:39:29,16.0,2019-07-23,23:39:29,0.0,0.0,0.0,75.0,51.0
8698,2019-07-23 23:44:48,16.0,2019-07-23,23:44:48,0.0,0.0,0.0,75.0,51.0
8699,2019-07-23 23:50:07,16.0,2019-07-23,23:50:07,0.0,0.0,0.0,75.0,51.0
8700,2019-07-23 23:55:29,16.0,2019-07-23,23:55:29,0.0,0.0,0.0,75.0,51.0


In [43]:
last_input=improved_old['time'].max()
last_input=last_input[:10]+" 00:00:00"
query = '''
SELECT time, val FROM `pi-iot-project-235918.home.sensors` 
where sensor = "Weight"
and time >'{}'
order by time desc 
'''.format(last_input)
# LIMIT 200
query

'\nSELECT time, val FROM `pi-iot-project-235918.home.sensors` \nwhere sensor = "Weight"\nand time >\'2019-07-24 00:00:00\'\norder by time desc \n'

In [44]:
df = gbq.read_gbq(query, dialect='standard' );

In [45]:
df.head()

,time,val
0,2019-07-26 13:02:30,37.0
1,2019-07-26 12:57:10,37.0
2,2019-07-26 12:51:53,37.0
3,2019-07-26 12:46:32,37.0
4,2019-07-26 12:41:08,37.0


In [46]:
df.count()

time    687
val     687
dtype: int64

In [47]:

df['time'] = df['time'].astype(str)

# Weight val cleaning

df=df[df.val > 0]
df=df[df.val < 200]

df['Day']=df['time'].str[:10]
df['Hour']=df['time'].str[10:]  

### Creating new Dataframe

In [49]:
# Converting function!!

# Data for the Days in the last query
Days=list(set(df['Day'])) 

#remove the last incomplete day...
improved=improved[~improved.Day.isin(Days)]


for day in Days:
    print(day)
    temp=df.copy(deep=True)

    temp=temp[temp.Day == day]
    temp = temp.sort_values(['time'])
    
    temp['delta']=0
    temp['eaten']=0
    temp['given']=0

    for i in range(1, len(temp)):
        value = temp['val'].iloc[i] - temp['val'].iloc[i-1]
        temp['delta'].iloc[i]=value
        if value < 0:
            temp['eaten'].iloc[i]= value * (-1)
        if value > 0:
            temp['given'].iloc[i]= value 
            
    
    temp['eaten_cum_Day']=temp['eaten']
    temp['given_cum_Day']=temp['given']

    for i in range(1, len(temp)):

        value = temp['eaten_cum_Day'].iloc[i-1] + temp['eaten_cum_Day'].iloc[i]
        temp['eaten_cum_Day'].iloc[i]=value

        value = temp['given_cum_Day'].iloc[i-1] + temp['given_cum_Day'].iloc[i]
        temp['given_cum_Day'].iloc[i]=value
    
    
    improved=improved.append(temp)

    

2019-07-25


/Users/Enzo.Calogero/anaconda3/envs/google/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


2019-07-24


/Users/Enzo.Calogero/anaconda3/envs/google/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


2019-07-26


/Users/Enzo.Calogero/anaconda3/envs/google/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [52]:
improved.head()

,time,val,Day,Hour,delta,eaten,given,eaten_cum_Day,given_cum_Day
0,2019-06-26 07:15:45,15.0,2019-06-26,07:15:45,0.0,0.0,0.0,0.0,0.0
1,2019-06-26 07:21:01,15.0,2019-06-26,07:21:01,0.0,0.0,0.0,0.0,0.0
2,2019-06-26 07:26:17,14.0,2019-06-26,07:26:17,-1.0,1.0,0.0,1.0,0.0
3,2019-06-26 07:31:33,14.0,2019-06-26,07:31:33,0.0,0.0,0.0,1.0,0.0
4,2019-06-26 07:36:53,14.0,2019-06-26,07:36:53,0.0,0.0,0.0,1.0,0.0


In [53]:
improved.to_csv('improved.csv', index=False)

### test

In [54]:
from datetime import datetime as dt

In [59]:
dt.today().strftime("%Y-%m-%d")

'2019-07-26'